In [1]:
import pandas as pd

In [5]:
val_context = pd.read_csv('val_context.csv', index_col=0)

In [6]:
val_context.head()

,context
index,
0,Hypertensive heart disease is the No. 1 cause ...
1,"Are you behind on your dental visits, and now ..."
2,"Delusional disorder, previously called paranoi..."
3,You did your best to protect yourself from bre...
4,Guttate psoriasis is a type of psoriasis that ...


In [40]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Load your 'val_context' dataframe
val_context = pd.read_csv('val_context.csv')

# Load the RoBERTa model and tokenizer
model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Encode context sentences
context_sentences = val_context['context'].tolist()
context_encodings = tokenizer(context_sentences, padding=True, truncation=True, return_tensors='pt', max_length=128, return_attention_mask=True)
with torch.no_grad():
    context_outputs = model(**context_encodings)
    context_embeddings = context_outputs.last_hidden_state.mean(dim=1) 

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
# Create a function to find the 10 most similar context sentences
def find_most_similar(sentence, context_embeddings, context_sentences):
    input_encoding = tokenizer(sentence, padding=True, truncation=True, return_tensors='pt', max_length=128, return_attention_mask=True)
    with torch.no_grad():
        input_output = model(**input_encoding)
        input_embedding = input_output.last_hidden_state.mean(dim=1) 

    similarities = cosine_similarity(context_embeddings, input_embedding)
    top_indices = np.argsort(similarities, axis=0)[-10:][::-1].flatten()
    top_contexts = [context_sentences[i] for i in top_indices]
    return top_contexts

In [48]:
# Example usage
sample_input_sentence = "tell me about adhd"
most_similar_context = find_most_similar(sample_input_sentence, context_embeddings, context_sentences)
print("Input sentence:", sample_input_sentence)
print("Most similar context sentences:")
for sentence in most_similar_context:
    print("-", sentence)

Input sentence: tell me about adhd
Most similar context sentences:
- When you think about diabetes drugs, you may think of insulin or other medications that you get from a shot or a pump. But there are others that you take as a pill or that you inhale. Your doctor will consider exactly what you need, which may include more than one type of diabetes medicine. The goal is to get your best blood sugar control, and the oral drugs do that in several ways. Acarbose ( Precose) How it works: Blocks enzymes that help digest starches, slowing the rise in blood sugar. It belongs to a group of drugs called âalpha-glucosidase inhibitors.â Side effects for these kinds of drugs include stomach upset (gas, diarrhea, nausea, cramps). Alogliptin ( Nesina) How it works: Boosts insulin levels when blood sugars are too high, and tells the liver to cut back on making sugars. Your doctor may call this type of drug a â DPP- IV inhibitor.â These drugs do not cause weight gain. You may take them alone o